# Bert text classification on SST2 using PyTorch





In [ ]:
!pip install -r requirements_notebook.txt

In [ ]:
import sys, os, shutil
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [ ]:
import sagemaker, boto3
from sagemaker import get_execution_role
sm_session = sagemaker.session.Session()


sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [ ]:
data_bucket = sm_session.default_bucket()

data_bucket_prefix = "bert-sst2-classify"

s3_uri_data = "s3://{}/{}/data".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train.csv")
s3_uri_val = "{}/{}".format(s3_uri_data, "dev.csv")

s3_uri_test = "{}/{}".format(s3_uri_data, "test.csv")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint".format(data_bucket, data_bucket_prefix)

In [ ]:
prepare_dataset = True

## Prepare dataset

In [ ]:
raw_data_dir = "tmp"
processed_out_dir = os.path.join(raw_data_dir, "processd")

if os.path.exists(processed_out_dir):
    shutil.rmtree(processed_out_dir)

os.makedirs(processed_out_dir, exist_ok=True)


In [ ]:
s3_uri_train

In [ ]:
from utils.sst2_split_utils import SST2SplitUtils
from s3_util import S3Util

if prepare_dataset:
    input_file = os.path.join(raw_data_dir , "datasetSentences.txt")
    split_file = os.path.join(raw_data_dir , "datasetSplit.txt")
    dictionary = os.path.join(raw_data_dir , "dictionary.txt")
    label_file = os.path.join(raw_data_dir , "sentiment_labels.txt")
    SST2SplitUtils().split( input_file, split_file, dictionary, label_file, processed_out_dir)
    
    S3Util().upload_file(os.path.join(processed_out_dir, "train.csv"), s3_uri_train )
    S3Util().upload_file(os.path.join(processed_out_dir, "dev.csv"), s3_uri_val )
    S3Util().upload_file(os.path.join(processed_out_dir, "test.csv"), s3_uri_test )
    
    

## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [ ]:
inputs_full =  {
    "train" : s3_uri_train,
    "val" : s3_uri_val,

}



inputs = inputs_full

In [ ]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [ ]:
instance_type = "ml.p3.2xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [ ]:
hp = {
"epochs" : 30,
"earlystoppingpatience" : 5,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"valfile" : s3_uri_val.split("/")[-1],
"datasetfactory":"datasets.sst2_dataset_factory.SST2DatasetFactory",
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2
}



In [ ]:
hp

In [ ]:
inputs

In [ ]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [ ]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample


In [ ]:
job_type = "bert-sst2-classification"
base_name = "{}".format(job_type)

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
                    entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir)

estimator.fit(inputs, wait=True)

## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     py_version = "py3",
                     framework_version='1.4.0',
                     entry_point='serve.py',
                     source_dir='src')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

### Invoke API

In [ ]:
import json


class TextSerDes:
    
     def serialize(self, x):
        data_bytes="\n".join(x).encode("utf-8")
        return data_bytes
    
     def deserialize(self, x, content_type):
        payload =   x.read().decode("utf-8")
        return json.loads(payload) 

In [ ]:
def chunk_predict(predictor, data, chunk_size=50):
    predictor.serializer = TextSerDes()
    predictor.deserializer = TextSerDes()
    
    result = []
    for i in range(0, len(data), chunk_size):
        
        re = predictor.predict(data[i:i+chunk_size],  initial_args={ "Accept":"text/json", "ContentType" : "text/csv" })
        result.extend(re)
    return result

In [ ]:
from s3_util import S3Util
import csv, io

def load_test_csv(s3_uri):
    os.path.join(processed_out_dir, "test.csv")
    data = S3Util().download_object(s3_uri).decode("utf-8")
    
    csv_reader = csv.reader(io.StringIO(data), delimiter='\t',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    actuals =[]
    inputs = []
    for r in csv_reader:
        label = r[1]
        text = r[0]
        
        inputs.append(text)
        actuals.append(label)
        
    return inputs, actuals
        

def write_predictions_csv(test_data, predictions, output_file):
    
    with open(output_file, "w") as f:
        csv_writer = csv.writer(f, delimiter='\t',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
            
        for d,p in zip(test_data, predictions):
            csv_writer.writerow([d,p])


In [ ]:

test_data, test_labels = load_test_csv(s3_uri_test)
response  = chunk_predict(predictor, test_data )
predictions_label = [ list(l.keys())[0] for l in response ]

write_predictions_csv(test_data, predictions_label, "sst2-output.csv" )

## Delete endpoint

In [ ]:
predictor.delete_endpoint()